In [23]:
!pip install -q geemap earthengine-api

In [24]:
# Set up EE
import ee
import geemap
ee.Authenticate()
ee.Initialize(project = "ee-frankfurt-fun-challenge")

In [25]:
# Get feature collection for German states
fc = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", "Germany"))

In [26]:
vis_params = {
    "color": "red",
    "fillColor": "00000000",
    "width": 3
}

Map = geemap.Map()
Map.centerObject(fc, 8)
Map.addLayer(fc.style(**vis_params), {}, "German States")
Map

Map(center=[51.05286656613144, 10.372108688819564], controls=(WidgetControl(options=['position', 'transparent_…

### Task 1: Locate Frankfurt/Main in Google Earth Engine and download a representative image of it

In [27]:
# Longitude and latitude of Franfurt
frankfurt_coords = [8.601211, 50.099498]
frankfurt = ee.Geometry.Point(frankfurt_coords)
frankfurt_square_region = frankfurt.buffer(10000).bounds()  # 10 km buffer for half side, then get bounds

In [28]:
# Load Sentinel-2 image collection, select Frankfurt and the desired dates, and choose the least cloudy image
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
  .filterBounds(frankfurt_square_region) \
  .filterDate("2023-01-01", "2023-12-31") \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
  .sort("CLOUDY_PIXEL_PERCENTAGE") \
  .first()

In [29]:
vis_params_frankfurt = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000
}
map_frankfurt = geemap.Map(center = frankfurt_coords[::-1], zoom = 12)
map_frankfurt.clear()
map_frankfurt.addLayer(collection.clip(frankfurt_square_region), vis_params_frankfurt, "Sentinel-2-Frankfurt")
map_frankfurt.addLayer(frankfurt_square_region, {}, "Frankfurt")
map_frankfurt

Map(center=[50.099498, 8.601211], crs={'name': 'EPSG3857', 'custom': False}, default_style=MapStyle(), draggin…

### Task 2: Calculate spectral indices in that image EVI, NEVI, MNDWI, RVI, NDVI, ARVI

In [30]:
# Calculate spectral indices from Sentinel-2 bands
def add_indices(image):
  nir = image.select("B8") # Near-infrared band
  red = image.select("B4") # Red band
  green = image.select("B3") # Green band
  blue = image.select("B2") # Blue band
  swir = image.select("B11") # Short-wave infrared band

  # Calculate indices
  ## EVI => Enhanced Vegetation Index
  evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': nir,
          'RED': red,
          'BLUE': blue
      }).rename('EVI')
  ## NEVI => Normalized Enhanced Vegetation Index
  nevi = image.expression(
      '(NIR - SWIR) / (NIR + SWIR)', {
          'NIR': nir,
          'SWIR': swir
      }).rename('NEVI')

  ## MNDWI => Modified Normalized Difference Water Index
  mndwi = image.expression(
      '(NIR - SWIR) / (NIR + SWIR)', {
          'NIR': nir,
          'SWIR': swir
      }).rename('MNDWI')

  ## RVI => Red Edge Vegetation Index
  rvi = image.expression(
      '(NIR - RED) / (NIR + RED)', {
          'NIR': nir,
          'RED': red
      }).rename('RVI')

  ## NDVI => Normalized Difference Vegetation Index
  ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")

  ## ARVI => Atmospherically Resistant Vegetation Index
  arvi = image.expression(
      '(NIR - RED) / (NIR + 0.5 * RED)', {
          'NIR': nir,
          'RED': red
      }).rename('ARVI')

  # Add indices to the image as new bands
  image = image.addBands([evi, nevi, mndwi, rvi, ndvi, arvi])
  return image

frankfurt_collection_with_indices = add_indices(collection.clip(frankfurt_square_region))

In [31]:
frankfurt_collection_with_indices

In [32]:
# Visualize spectral indices
map_frankfurt.addLayer(frankfurt_collection_with_indices.select("EVI"), {"min": -1, "max": 1, "palette": ["white", "blue"]}, "EVI")
map_frankfurt

Map(center=[50.099498, 8.601211], crs={'name': 'EPSG3857', 'custom': False}, default_style=MapStyle(), draggin…

In [33]:
#NEVI
map_frankfurt.addLayer(frankfurt_collection_with_indices.select("NDVI"), {"min": -1, "max": 1, "palette": ["white", "green"]}, "NDVI")
map_frankfurt

Map(bottom=177871.0, center=[50.1051665428782, 8.58135223388672], crs={'name': 'EPSG3857', 'custom': False}, d…